## Colorado Elk Tag Application Data Collection

Raw Data that was ingested can be found [Here](https://cpw.state.co.us/thingstodo/Pages/Statistics-Elk.aspx)

### Packages

In [1]:
import pandas as pd
import requests
import pdfplumber
import os
import shutil
from io import StringIO
from bs4 import BeautifulSoup
from re import search

In [2]:
#Webscraping URL's Related to Elk Draw Recaps
#Scraping html
url = 'https://cpw.state.co.us/thingstodo/Pages/Statistics-Elk.aspx'
r = requests.get(url)
html_doc = r.text
soup = BeautifulSoup(html_doc)

#isolating href files to a list
site_URLs = []
for link in soup.find_all('a'):
    site_URLs.append((link.get('href')))

#Converting to DataFrame and Filtering for information on Draw Recap Statistics
elk_URL_DF_Raw = pd.DataFrame(site_URLs,columns =['URL'])
elk_URL_DF_Filtered = elk_URL_DF_Raw.loc[elk_URL_DF_Raw['URL'].str.contains('/Documents/Hunting/BigGame/Statistics/ELK',case=False,na=False)]
elk_URL_DF_Filtered = elk_URL_DF_Filtered.loc[elk_URL_DF_Raw['URL'].str.contains('ElkDrawRecap.pdf',case=False,na=False)]
elk_URL_DF_Filtered['URL'] = 'https://cpw.state.co.us' + elk_URL_DF_Filtered['URL'].astype(str)
elk_URL_DF_Filtered.reset_index(inplace = True, drop=True)

print(elk_URL_DF_Filtered['URL'])

0    https://cpw.state.co.us/Documents/Hunting/BigG...
1    https://cpw.state.co.us/Documents/Hunting/BigG...
2    https://cpw.state.co.us/Documents/Hunting/BigG...
3    https://cpw.state.co.us/Documents/Hunting/BigG...
4    https://cpw.state.co.us/Documents/Hunting/BigG...
5    https://cpw.state.co.us/Documents/Hunting/BigG...
6    https://cpw.state.co.us/Documents/Hunting/BigG...
7    https://cpw.state.co.us/Documents/Hunting/BigG...
Name: URL, dtype: object


## Functions

In [3]:
def download_file(url):
    local_filename = url.split('/')[-1]

    with requests.get(url) as r:
        with open(local_filename, 'wb') as f:
            f.write(r.content)
        
    return local_filename

def check_space(string):
    """Function that returns the number of strings of the inputted string"""
    count = 0
    for i in string:
        if i == " ":
            count += 1
    return count

def find_hunt_code(df,page_No,pdf_length):
    """
    Input: Dataframe from the pdf page

    Output: Huntcode as a string
    """
    # Utilize a regular expression to find the Hunt Code
    Hunt_Code_Search = search('[A-Z]{2}\d{3}[A-Z]{1}\d{1}[A-Z]{1}',df.iloc[2,0])

    try:
        Hunt_Code = Hunt_Code_Search.group(0)
        Hunt_Code_Storage = Hunt_Code_Search.group(0)
    except:
        Hunt_Code = None
    
    return Hunt_Code

def strip_whitespace(df):
    """
    Input: Dataframe from the pdf page
    
    Output: Dataframe with whitespaces stripped
    """
    # Verifying that the dtypes are objects
    df_object = df.select_dtypes(['object'])

    #Strip all white spaces
    df[df_object.columns] = df_object.apply(lambda x: x.str.strip())

    return df

def find_preference_point_table(df):
    """
    Input: Dataframe from the pdf page

    Output: Preference Point table or Null
    """
    column_name = df.columns[0]

    try:
        PP_Start = df.loc[df[column_name].str.contains('Choice Preference',na=False,case=False)]
        #PP_Start = df.loc[df['                  Colorado Parks and Wildlife   Draw Recap'].str.contains('Choice Preference',na=False,case=False)]
        Preference_Points = df.iloc[PP_Start.index[0]+1:]
    except:
        try:
            #There's an extra space between Choice and Preference in some cases
            PP_Start = df.loc[df[column_name].str.contains('Choice  Preference',na=False,case=False)]
            Preference_Points = df.iloc[PP_Start.index[0]+1:]
        except:
            #PP_Start = df.loc[df['                  Colorado Parks and Wildlife   Draw Recap'].str.contains('Page',na=False,case=False)]
            PP_Start = df.loc[df[column_name].str.contains('Page',na=False,case=False)]
            Preference_Points = df.iloc[PP_Start.index[0]+1:]
    
    return Preference_Points

def choice_finder(df):
    """
    Input: Pre processing Preference Points Dataframe
    
    Output: Isolated Column
    """
    column_name = df.columns[0]

    #Handle isolated instances where an extra space was added (i.e. 2020)
    df[column_name]= df[column_name].str.replace("  "," ")

    #Check how many spaces are in the strings to see if its in the standardized format, or if an extra character is there
    df['Choice Finder'] = df["Preference Points Table Buffer"].apply(lambda x: check_space(x))

    #Check if the number of spaces matches the number in the standardized format and the format with the choice included
    df = df.loc[(df['Choice Finder'] == 15) | (df['Choice Finder'] == 13)]
    df.reset_index(inplace=True, drop=True)

    #Isolate the choice made in the draw, which is indicated based on a string length of 15
    df['Choice'] = [x[:1] if y == 15 else None for x,y in zip(df['Preference Points Table Buffer'],df['Choice Finder'])]
    #df['Choice'] = [x[:1] if y == 15 else None for x,y in zip(df[column_name],df['Choice Finder'])]
    Choice_Index = df[df['Choice'].notnull()].index
    df.bfill(axis='rows',inplace=True)
    df.ffill(axis='rows',inplace=True)

    #Table to merge choices by index after the preference points transpormation is complete
    Choice_Merge = df['Choice']

    #Restucting rows that had the choice in with the preference points values
    df['Preference Points Table Buffer'] = [x[2:] if y == 15 else x for x,y in zip(df['Preference Points Table Buffer'],df['Choice Finder'])]
    #df['Preference Points Table Buffer'] = [x[2:] if y == 15 else x for x,y in zip(column_name,df['Choice Finder'])]

    return df, Choice_Merge, Choice_Index

def preference_points_clean_up(df,Choice_Index):
    """
    Input: Preference Points Buffer 2 and the Choice Index
        
    Output: Cleaned up preference points table
    """

    #Expand the restructured preference point dataframe, so it matches the format in the pdf
    df_Expanded = df.iloc[:,0].str.split(' ',expand=True)

    try:
        df_Expanded = df_Expanded[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]]

        #For loop to handle the choice being in the draw portion of the preference point table
        for j in range(7,14):
            df_Expanded.iloc[Choice_Index,j] = df_Expanded.iloc[Choice_Index,j+1]

        #Removes the blank column at the end
        del df_Expanded[14]

        #Remove the redundent preference point column
        del df_Expanded[7]

    except:
        df_Expanded = df_Expanded[[0,1,2,3,4,5,6,7,8,9,10,11,12,13]]

        #Remove the redundent preference point column
        del df_Expanded[7]

    # Remove any residule components that aren't an integer in the preference points columns
    df_Expanded[0] = pd.to_numeric(df_Expanded[0],errors = 'coerce')
    df_Expanded.dropna(inplace = True)
    df_Expanded[0] = pd.to_numeric(df_Expanded[0],downcast="integer")

    #Rename the columns, so they correlate with the pdf format
    df_Expanded.columns = ['Preference Points','A-Adult-Res','A-Adult-NonRes','A-Youth-Res','A-Youth-NonRes','A-Landowner(LPP)-Unrestricted' \
    ,'A-Landownder(LPP)-Restricted','D-Adult-Res','D-Adult-NonRes','D-Youth-Res','D-Youth-NonRes'
    ,'D-Landowner(LPP)-Unrestricted','D-Landownder(LPP)-Restricted']

    return df_Expanded

def preference_points_finalize(df, Choice_Merge, Hunt_Code, Draw_Year):
    """
    Input: Preference Points Expanded DataFrame, Choice Merge Dataframe, Hunt_Code, and Draw Year

    Output: Applicants and Drew Dataframes accordingly
    """
    #Isolate the columns appliable to the draw applicants
    Applicant_Preference_Points_Buffer = df[['Preference Points','A-Adult-Res','A-Adult-NonRes','A-Youth-Res','A-Youth-NonRes','A-Landowner(LPP)-Unrestricted' \
    ,'A-Landownder(LPP)-Restricted']]

    #Merge the Choice on index; creating the applicant and successful draw dataframe
    Applicant_Preference_Points = Applicant_Preference_Points_Buffer.merge(Choice_Merge, left_index=True, right_index=True)

    Draw_Preference_Points_Buffer = df[['Preference Points','D-Adult-Res','D-Adult-NonRes','D-Youth-Res','D-Youth-NonRes'
    ,'D-Landowner(LPP)-Unrestricted','D-Landownder(LPP)-Restricted']]

    Draw_Preference_Points = Draw_Preference_Points_Buffer.merge(Choice_Merge, left_index=True,right_index=True)

    #Add the Hunt Code and Draw year to the dataframes
    Applicant_Preference_Points['Hunt Code'] = Hunt_Code
    Applicant_Preference_Points['Year'] = Draw_Year
    Draw_Preference_Points['Hunt Code'] = Hunt_Code
    Draw_Preference_Points['Year'] = Draw_Year

    #Create a Primary Key for the dataframe
    Applicant_Preference_Points['Hunt Key'] = [x + '-' + y + '-' + str(z) for x,y,z in zip(Applicant_Preference_Points['Hunt Code'], \
        Applicant_Preference_Points['Year'],Applicant_Preference_Points['Preference Points'])]
    Draw_Preference_Points['Hunt Key'] = [x + '-' + y + '-' + str(z) for x,y,z in zip(Draw_Preference_Points['Hunt Code'],Draw_Preference_Points['Year'] \
        ,Draw_Preference_Points['Preference Points'])]

    return Applicant_Preference_Points, Draw_Preference_Points

In [81]:
elk_URL_DF_Filtered['URL'][7]

'https://cpw.state.co.us/Documents/Hunting/BigGame/Statistics/Elk/2015ElkDrawRecap.pdf'

## PDF Scraping Section

In [ ]:
#for i in range(0,len(elk_URL_DF_Filtered)):
for i in range(7,8):

    stats_url = download_file(elk_URL_DF_Filtered['URL'][i])

    # Instanciate empty dataframes for the consolidated output of processed pages
    All_Applicant_Preference_Points = pd.DataFrame()
    All_Draw_Preference_Points = pd.DataFrame()
    Pages_with_issues = []

    with pdfplumber.open(stats_url) as pdf:
        number_of_pages = len(pdf.pages)
        for j in range(2,number_of_pages):
            # Export the pdf page's raw text as a dataframe
            page = pdf.pages[j]
            text = page.extract_text()
            df = pd.read_csv(StringIO(text))

            #Length of less than 8 skips pages that only have the portions of the summary table below the preference point table
            if len(df) > 8:
                try:
                    #Find the Hunt Code on the page and leave it the same if there isn't one on the page
                    Hunt_Code_Buffer = find_hunt_code(df,j,number_of_pages)
                    Hunt_Code = Hunt_Code if Hunt_Code_Buffer is None else Hunt_Code_Buffer

                    #print('Hunt Code Found Successfully...')

                    # Utilize a regular expression to find the Year of the Draw Recap
                    Draw_Year_Search = search('\d{4}',df.iloc[0,0])
                    Draw_Year = Draw_Year_Search.group(0)

                    #print('Year Found Successfully...')

                    #Strip whitespaces
                    df_stripped = strip_whitespace(df)

                    #print('Whitespaces Stripped Successfully..')

                    #Isolating where the preference points portion of the dataframe starts
                    Preference_Points_Buffer = find_preference_point_table(df_stripped)

                    #print('Preference Point Table Found Successfully...')

                    Preference_Points_Buffer.reset_index(inplace=True, drop=True)
                    Preference_Points_Buffer.columns = ["Preference Points Table Buffer"]

                    #print('Index reset and column renamed successfully...')

                    #Isolate the Choice and reformat to a standardize format for separating the preference points
                    Preference_Points_Buffer2, Choice_Merge, Choice_Index = choice_finder(Preference_Points_Buffer)

                    #print('Choice found successfully...')

                    # Reformat Preference Point DataFrame, so it's easier to interpret
                    Preference_Points_Expanded = preference_points_clean_up(Preference_Points_Buffer2, Choice_Index)

                    #print('Preference Point table cleaned up successfully...')

                    #Perform final clean-up to and segregation
                    Applicant_Preference_Points, Draw_Preference_Points = preference_points_finalize(Preference_Points_Expanded, Choice_Merge, Hunt_Code, Draw_Year)

                    #print('Preference Point Clean-up Completed Successfully')

                    #Append to a generalized Dataframe for multiple pages processed
                    All_Applicant_Preference_Points = All_Applicant_Preference_Points.append(Applicant_Preference_Points)
                    All_Draw_Preference_Points = All_Draw_Preference_Points.append(Draw_Preference_Points)

                    #print('Appended to consolidated dataframe successfully')
                except:
                    Pages_with_issues.append(j)
                    continue
            else:
                continue

    All_Applicant_Preference_Points.to_excel('Output-Data\\Applicant-Data\\Dirty\\'+ Draw_Year+'-All-Applicant-Preference-Points.xlsx')
    All_Draw_Preference_Points.to_excel('Output-Data\\Draw-Data\\Dirty\\'+Draw_Year+'-All-Draw-Preference-Points.xlsx')
    
    Pages_With_Issues_DF = pd.DataFrame(Pages_with_issues, columns=['Pages'])
    if len(Pages_With_Issues_DF) > 0:
        Pages_With_Issues_DF.to_excel('Output-Data\\Pages-with-Issues\\'+ Draw_Year+'-Pages-With-Issues.xlsx')


## Input Data Directory Clean-up

In [31]:
# Get current directory
directory = os.getcwd()
    
#Iterate over the directory looking for the downloaded input pdf documents and put into a separate directory
for file in os.listdir(directory):
     filename = os.fsdecode(file)
     if filename.endswith(".pdf"): 
        original = filename
        target = 'Input-Data\\'+filename

        shutil.move(original, target)
        continue
     else:
         continue

## Cleaning up data types and Choice column for applicants

In [2]:
# Set directory to iterate throughout all the the files in
directory = 'Output-Data\\Applicant-Data\\Dirty\\'
    
#Iterate over all the files in the directory and perfom transformations if it's an .xlsx file
for file in os.listdir(directory):
     filename = os.fsdecode(file)
     if (filename.endswith(".xlsx")) & (filename[0:3] != "Tot"):
      
      # Import File as a dataframe
      df = pd.read_excel(directory+file)

      # Replace the "-" placeholder used by CPW with 0
      df = df.replace("-",0)

      # Dictionary to map the columns as the correct data type
      convert_dict ={'Preference Points':int
      ,'A-Adult-Res':int
      ,'A-Adult-NonRes':int
      ,'A-Youth-Res':int
      ,'A-Youth-NonRes':int
      ,'A-Landowner(LPP)-Unrestricted':int
      ,'A-Landownder(LPP)-Restricted':int}

      # Remap dataframe with the correct data type
      df= df.astype(convert_dict)

      #Loop to check the preference points and hunt code, then assign the correct Choice
      Choice = 1
      for i in range (1,len(df)):
         if df.iloc[i,9] == df.iloc[i-1,9]:
            if df.iloc[i,0] == df.iloc[i-1,0]+1:
               df.iloc[i,8] = Choice
            else:
               Choice +=1
               df.iloc[i,8] = Choice
         else:
            Choice = 1
            df.iloc[i,8] = Choice

      df['Choice']=df['Choice'].fillna(0).astype(int)

      #Delete old Choice column
      del df[df.columns[0]]

      #Select only Choice 1 applicants
      output_df  = df.loc[df['Choice']==1]

      # Output the re-formatted dataframe
      output_df.to_excel('Output-Data\\Applicant-Data\\Cleaned\\'+filename)

      continue
     else:
      continue

 ## Cleaning up data types and choice column for draws

In [4]:
# Set directory to iterate throughout all the the files in
directory = 'Output-Data\\Draw-Data\\Dirty\\'
    
#Iterate over all the files in the directory and perfom transformations if it's an .xlsx file    
for file in os.listdir(directory):
     filename = os.fsdecode(file)
     if (filename.endswith(".xlsx")) & (filename[0:3] != "Tot"):

      # Import File as a dataframe
      df = pd.read_excel(directory+file)

      # Replace the "-" placeholder used by CPW with 0
      df = df.replace("-",0)

      # Dictionary to map the columns as the correct data type
      convert_dict ={'Preference Points':int
      ,'D-Adult-Res':int
      ,'D-Adult-NonRes':int
      ,'D-Youth-Res':int
      ,'D-Youth-NonRes':int
      ,'D-Landowner(LPP)-Unrestricted':int
      ,'D-Landownder(LPP)-Restricted':int}

      # Remap dataframe with the correct data type
      df= df.astype(convert_dict)

      #Loop to check the preference points and hunt code, then assign the correct Choice
      Choice = 1
      for i in range (1,len(df)):
         if df.iloc[i,9] == df.iloc[i-1,9]:
            if df.iloc[i,0] == df.iloc[i-1,0]+1:
               df.iloc[i,8] = Choice
            else:
               Choice +=1
               df.iloc[i,8] = Choice
         else:
            Choice = 1
            df.iloc[i,8] = Choice

      df['Choice']=df['Choice'].fillna(0).astype(int)

      #Delete old Choice column
      del df[df.columns[0]]

      #Select only Choice 1 applicants
      output_df  = df.loc[df['Choice']==1]

      # Output the re-formatted dataframe
      output_df.to_excel('Output-Data\\Draw-Data\\Cleaned\\'+filename)

      continue
     else:
      continue

### Find the total tags awarded by year, hunt code, and resident type

In [3]:
# Set directory to iterate throughout all the the files in
directory = 'Output-Data\\Draw-Data\\Cleaned\\'
    
#Iterate over all the files in the directory and perfom transformations if it's an .xlsx file
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if (filename.endswith(".xlsx")) & (filename[0:3] != "Tot"):
        df_total_tags_init = pd.read_excel(directory+file)

        #Dictionary to add how many tags were awarded for each sub category
        Total_Tags_Dict = {}
        for i in range (0,len(df_total_tags_init)):
            if i == 0:
                AR_Total_Tags_Given = df_total_tags_init.iloc[i,2]
                ANR_Total_Tags_Given = df_total_tags_init.iloc[i,3]
                YR_Total_Tags_Given = df_total_tags_init.iloc[i,4]
                YNR_Total_Tags_Given = df_total_tags_init.iloc[i,5]
                LOU_Total_Tags_Given = df_total_tags_init.iloc[i,6]
                LOR_Total_Tags_Given = df_total_tags_init.iloc[i,7]
            elif df_total_tags_init.iloc[i,9] == df_total_tags_init.iloc[i-1,9]:
                AR_Total_Tags_Given += df_total_tags_init.iloc[i,2]
                ANR_Total_Tags_Given += df_total_tags_init.iloc[i,3]
                YR_Total_Tags_Given += df_total_tags_init.iloc[i,4]
                YNR_Total_Tags_Given += df_total_tags_init.iloc[i,5]
                LOU_Total_Tags_Given += df_total_tags_init.iloc[i,6]
                LOR_Total_Tags_Given += df_total_tags_init.iloc[i,7]
            else:
                Total_Tags_Dict[df_total_tags_init.iloc[i-1,9]] = {
                    'Adult-Res':AR_Total_Tags_Given
                    ,'Adult-Non Res':ANR_Total_Tags_Given
                    ,'Youth-Res':YR_Total_Tags_Given
                    ,'Youth-Non Res':YNR_Total_Tags_Given
                    ,'Landowner-Unrestricted':LOU_Total_Tags_Given
                    ,'Landownder-Restricted':LOR_Total_Tags_Given
                }
                AR_Total_Tags_Given = df_total_tags_init.iloc[i,2]
                ANR_Total_Tags_Given = df_total_tags_init.iloc[i,3]
                YR_Total_Tags_Given = df_total_tags_init.iloc[i,4]
                YNR_Total_Tags_Given = df_total_tags_init.iloc[i,5]
                LOU_Total_Tags_Given = df_total_tags_init.iloc[i,6]
                LOR_Total_Tags_Given = df_total_tags_init.iloc[i,7]

        # Convert Dictionary to a dataframe
        Total_Tags_Awarded =pd.DataFrame.from_dict(Total_Tags_Dict,orient='index')

        # Sum each column to get the total number of tags awarded
        Total_Tags_Awarded['Total Tags'] = [a+b+c+d+e+f for a,b,c,d,e,f in zip(Total_Tags_Awarded['Adult-Res'] \
            ,Total_Tags_Awarded['Adult-Non Res']
            ,Total_Tags_Awarded['Youth-Res']
            ,Total_Tags_Awarded['Youth-Non Res']
            ,Total_Tags_Awarded['Landowner-Unrestricted']
            ,Total_Tags_Awarded['Landownder-Restricted'])]
        
        # Create a Year Column based on the filename string
        Total_Tags_Awarded['Year'] = filename[0:4]

        # Create a Hunt Code Column
        Total_Tags_Awarded['Hunt Code'] = Total_Tags_Awarded.index

        #Create a column for a the primary key
        Total_Tags_Awarded['Primary Key'] = [x +'-'+ y for x,y in zip(Total_Tags_Awarded['Year'], Total_Tags_Awarded.index)]

        #Output the Total Tags awarded dataframe
        Total_Tags_Awarded.to_excel('Output-Data\\Total-Awarded\\'+filename[0:4]+'-Total-Tags-Awarded.xlsx')
        continue
    else:
        continue

### Find the total quantity of applicants by year, hunt code, and resident type

In [5]:
# Set directory to iterate throughout all the the files in
directory = 'Output-Data\\Applicant-Data\\Cleaned\\'
    
#Iterate over all the files in the directory and perfom transformations if it's an .xlsx file
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if (filename.endswith(".xlsx")) & (filename[0:3] != "Tot"):
        df_total_apps_init = pd.read_excel(directory+file)

        #Dictionary to add how many tags were awarded for each sub category
        Total_Apps_Dict = {}
        for i in range (0,len(df_total_apps_init)):
            if i == 0:
                AR_Total_Apps = df_total_apps_init.iloc[i,2]
                ANR_Total_Apps = df_total_apps_init.iloc[i,3]
                YR_Total_Apps = df_total_apps_init.iloc[i,4]
                YNR_Total_Apps = df_total_apps_init.iloc[i,5]
                LOU_Total_Apps = df_total_apps_init.iloc[i,6]
                LOR_Total_Apps = df_total_apps_init.iloc[i,7]
            elif df_total_tags_init.iloc[i,9] == df_total_tags_init.iloc[i-1,9]:
                AR_Total_Apps += df_total_apps_init.iloc[i,2]
                ANR_Total_Apps += df_total_apps_init.iloc[i,3]
                YR_Total_Apps += df_total_apps_init.iloc[i,4]
                YNR_Total_Apps += df_total_apps_init.iloc[i,5]
                LOU_Total_Apps += df_total_apps_init.iloc[i,6]
                LOR_Total_Apps += df_total_apps_init.iloc[i,7]
            else:
                Total_Apps_Dict[df_total_apps_init.iloc[i-1,9]] = {
                    'Adult-Res':AR_Total_Apps
                    ,'Adult-Non Res':ANR_Total_Apps
                    ,'Youth-Res':YR_Total_Apps
                    ,'Youth-Non Res':YNR_Total_Apps
                    ,'Landowner-Unrestricted':LOU_Total_Apps
                    ,'Landownder-Restricted':LOR_Total_Apps
                }
                AR_Total_Apps = df_total_apps_init.iloc[i,2]
                ANR_Total_Apps = df_total_apps_init.iloc[i,3]
                YR_Total_Apps = df_total_apps_init.iloc[i,4]
                YNR_Total_Apps = df_total_apps_init.iloc[i,5]
                LOU_Total_Apps = df_total_apps_init.iloc[i,6]
                LOR_Total_Apps = df_total_apps_init.iloc[i,7]

        # Convert Dictionary to a dataframe
        Total_Apps =pd.DataFrame.from_dict(Total_Apps_Dict,orient='index')

        # Sum each column to get the total number of tags awarded
        Total_Apps['Total Applications'] = [a+b+c+d+e+f for a,b,c,d,e,f in zip(Total_Apps['Adult-Res'] \
            ,Total_Apps['Adult-Non Res']
            ,Total_Apps['Youth-Res']
            ,Total_Apps['Youth-Non Res']
            ,Total_Apps['Landowner-Unrestricted']
            ,Total_Apps['Landownder-Restricted'])]
        
        # Create a Year Column based on the filename string
        Total_Apps['Year'] = filename[0:4]

        # Create a Hunt Code Column
        Total_Apps['Hunt Code'] = Total_Apps.index

        #Create a column for a the primary key
        Total_Apps['Primary Key'] = [x +'-'+ y for x,y in zip(Total_Apps['Year'], Total_Apps.index)]

        #Output the Total Tags awarded dataframe
        Total_Apps.to_excel('Output-Data\\Total-Applied\\'+filename[0:4]+'-Total-Applied.xlsx')
        continue
    else:
        continue

### Find the minimum preference points to draw tags for hunt codes and resident type

In [6]:
directory = 'Output-Data\\Draw-Data\\Cleaned\\'
    
#Iterate over all the files in the directory and perfom transformations if it's an .xlsx file
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if (filename.endswith(".xlsx")) & (filename[0:3] != "Tot"):
        df_min_pts_init = pd.read_excel(directory+file)

        #Dictionary to add how many tags were awarded for each sub category
        Min_Pref_Dict = {}
        for i in range (0,len(df_min_pts_init)):
            if i == 0:
                if (df_min_pts_init.iloc[i,2] > 0):
                    AR_Min_Pts = df_min_pts_init.iloc[i,1]
                else:
                    AR_Min_Pts = 99
                
                if (df_min_pts_init.iloc[i,3] > 0):
                    ANR_Min_Pts = df_min_pts_init.iloc[i,1]
                else:
                    ANR_Min_Pts = 99

                if (df_min_pts_init.iloc[i,4] > 0):
                    YR_Min_Pts = df_min_pts_init.iloc[i,1]
                else:
                    YR_Min_Pts = 99

                if (df_min_pts_init.iloc[i,5] > 0):
                    YNR_Min_Pts = df_min_pts_init.iloc[i,1]
                else:
                    YNR_Min_Pts = 99

                if (df_min_pts_init.iloc[i,6] > 0):
                    LOU_Min_Pts = df_min_pts_init.iloc[i,1]
                else:
                    LOU_Min_Pts = 99

                if (df_min_pts_init.iloc[i,7] > 0):
                    LOR_Min_Pts = df_min_pts_init.iloc[i,1]
                else:
                    LOR_Min_Pts = 99

            elif (df_min_pts_init.iloc[i,9] == df_min_pts_init.iloc[i-1,9]):
                if ((df_min_pts_init.iloc[i,2] > 0) & (AR_Min_Pts == 99)):
                    AR_Min_Pts = df_min_pts_init.iloc[i,1]
                elif ((df_min_pts_init.iloc[i,2] > 0) & (df_min_pts_init.iloc[i,1] < AR_Min_Pts)):
                    AR_Min_Pts = df_min_pts_init.iloc[i,1]
                
                if ((df_min_pts_init.iloc[i,3] > 0) & (ANR_Min_Pts == 99)):
                    ANR_Min_Pts = df_min_pts_init.iloc[i,1]
                elif ((df_min_pts_init.iloc[i,3] > 0) & (df_min_pts_init.iloc[i,1] < ANR_Min_Pts)):
                    ANR_Min_Pts = df_min_pts_init.iloc[i,1]

                if ((df_min_pts_init.iloc[i,4] > 0) & (YR_Min_Pts == 99)):
                    YR_Min_Pts = df_min_pts_init.iloc[i,1]
                elif ((df_min_pts_init.iloc[i,4] > 0) & (df_min_pts_init.iloc[i,1] < YR_Min_Pts)):
                    YR_Min_Pts = df_min_pts_init.iloc[i,1]

                if ((df_min_pts_init.iloc[i,5] > 0) & (YNR_Min_Pts == 99)):
                    YNR_Min_Pts = df_min_pts_init.iloc[i,1]
                elif ((df_min_pts_init.iloc[i,5] > 0) & (df_min_pts_init.iloc[i,1] < YNR_Min_Pts)):
                    YNR_Min_Pts = df_min_pts_init.iloc[i,1]

                if ((df_min_pts_init.iloc[i,6] > 0) & (LOU_Min_Pts == 99)):
                    LOU_Min_Pts = df_min_pts_init.iloc[i,1]
                elif ((df_min_pts_init.iloc[i,6] > 0) & (df_min_pts_init.iloc[i,1] < LOU_Min_Pts)):
                    LOU_Min_Pts = df_min_pts_init.iloc[i,1]

                if ((df_min_pts_init.iloc[i,7] > 0) & (LOR_Min_Pts == 99)):
                    LOR_Min_Pts = df_min_pts_init.iloc[i,1]
                elif ((df_min_pts_init.iloc[i,7] > 0) & (df_min_pts_init.iloc[i,1] < LOR_Min_Pts)):
                    LOR_Min_Pts = df_min_pts_init.iloc[i,1]

            else:
                Min_Pref_Dict[df_min_pts_init.iloc[i-1,9]] = {
                    'Adult-Res':AR_Min_Pts
                    ,'Adult-Non Res':ANR_Min_Pts
                    ,'Youth-Res':YR_Min_Pts
                    ,'Youth-Non Res':YNR_Min_Pts
                    ,'Landowner-Unrestricted':LOU_Min_Pts
                    ,'Landownder-Restricted':LOR_Min_Pts
                }
                if (df_min_pts_init.iloc[i,2] > 0):
                    AR_Min_Pts = df_min_pts_init.iloc[i,1]
                else:
                    AR_Min_Pts = 99
                
                if (df_min_pts_init.iloc[i,3] > 0):
                    ANR_Min_Pts = df_min_pts_init.iloc[i,1]
                else:
                    ANR_Min_Pts = 99

                if (df_min_pts_init.iloc[i,4] > 0):
                    YR_Min_Pts = df_min_pts_init.iloc[i,1]
                else:
                    YR_Min_Pts = 99

                if (df_min_pts_init.iloc[i,5] > 0):
                    YNR_Min_Pts = df_min_pts_init.iloc[i,1]
                else:
                    YNR_Min_Pts = 99

                if (df_min_pts_init.iloc[i,6] > 0):
                    LOU_Min_Pts = df_min_pts_init.iloc[i,1]
                else:
                    LOU_Min_Pts = 99

                if (df_min_pts_init.iloc[i,7] > 0):
                    LOR_Min_Pts = df_min_pts_init.iloc[i,1]
                else:
                    LOR_Min_Pts = 99

        # Convert Dictionary to a dataframe
        Total_Min_Pref_Awarded =pd.DataFrame.from_dict(Min_Pref_Dict,orient='index')

        # Create a Year Column based on the filename string
        Total_Min_Pref_Awarded['Year'] = filename[0:4]

        #Create a column for a the primary key
        Total_Min_Pref_Awarded['Primary Key'] = [x +'-'+ y for x,y in zip(Total_Min_Pref_Awarded['Year'], Total_Min_Pref_Awarded.index)]

        # Replace the 99 placeholder with None
        Total_Min_Pref_Awarded['Adult-Res'].replace({99: None},inplace=True)
        Total_Min_Pref_Awarded['Adult-Non Res'].replace({99: None},inplace=True)
        Total_Min_Pref_Awarded['Youth-Res'].replace({99: None},inplace=True)
        Total_Min_Pref_Awarded['Youth-Non Res'].replace({99: None},inplace=True)
        Total_Min_Pref_Awarded['Landowner-Unrestricted'].replace({99: None},inplace=True)
        Total_Min_Pref_Awarded['Landownder-Restricted'].replace({99: None},inplace=True)

        #Output the Total Tags awarded dataframe
        Total_Min_Pref_Awarded.to_excel('Output-Data\\Minimum-Preference-Points\\'+filename[0:4]+'-Min-Pref-Points.xlsx')
        continue
    else:
        continue

### Consolidate Data for each output into a master source for all date ranges

In [7]:
directory = 'Output-Data\\Applicant-Data\\Cleaned\\'

All_Applicants = pd.DataFrame()

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if (filename.endswith(".xlsx")) & (filename[0:3] != "Tot"):
        df_Applicants = pd.read_excel(directory+file)
        All_Applicants = pd.concat([All_Applicants,df_Applicants])
        continue
    else:
        continue

del All_Applicants[All_Applicants.columns[0]]

All_Applicants['Secondary Key'] = [str(x) + "-" + y for x,y in zip(All_Applicants['Year'],All_Applicants['Hunt Code'])]

All_Applicants.to_excel('Output-Data\\Applicant-Data\\Cleaned\\Total-All-Applicant-Preference_Points.xlsx')

In [8]:
directory = 'Output-Data\\Draw-Data\\Cleaned\\'

All_Draw = pd.DataFrame()

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if (filename.endswith(".xlsx")) & (filename[0:3] != "Tot"):
        df_Draw = pd.read_excel(directory+file)
        All_Draw = pd.concat([All_Draw,df_Draw])
        continue
    else:
        continue

del All_Draw[All_Draw.columns[0]]

All_Draw['Secondary Key'] = [str(x) + "-" + y for x,y in zip(All_Draw['Year'],All_Draw['Hunt Code'])]

All_Draw.to_excel('Output-Data\\Draw-Data\\Cleaned\\Total-All-Draw-Preference_Points.xlsx')

In [9]:
directory = 'Output-Data\\Total-Awarded\\'

All_Awarded = pd.DataFrame()

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if (filename.endswith(".xlsx")) & (filename[0:3] != "All"):
        df_Total_Awarded = pd.read_excel(directory+file)
        All_Awarded = pd.concat([All_Awarded,df_Total_Awarded])
        continue
    else:
        continue

del All_Awarded[All_Awarded.columns[0]]

All_Awarded.to_excel('Output-Data\\Total-Awarded\\All-Total-Tags-Awarded.xlsx')

In [10]:
directory = 'Output-Data\\Total-Applied\\'

All_Applied = pd.DataFrame()

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if (filename.endswith(".xlsx")) & (filename[0:3] != "All"):
        df_Total_Applied = pd.read_excel(directory+file)
        All_Applied = pd.concat([All_Applied,df_Total_Applied])
        continue
    else:
        continue

del All_Applied[All_Applied.columns[0]]

All_Applied.to_excel('Output-Data\\Total-Applied\\All-Total-Applied.xlsx')

In [ ]:
directory = 'Output-Data\\Minimum-Preference-Points\\'

All_Min_Pref = pd.DataFrame()

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if (filename.endswith(".xlsx")) & (filename[0:3] != "All"):
        df_All_Min_Pref_Points = pd.read_excel(directory+file)
        df_All_Min_Pref_Points.reset_index(inplace=False,drop=False)
        All_Min_Pref = pd.concat([All_Min_Pref,df_All_Min_Pref_Points])
        All_Min_Pref.reset_index(inplace=True, drop=True)
        continue
    else:
        continue

All_Min_Pref.rename(columns = {'Unnamed: 0':"Hunt Code"},inplace=True)

All_Min_Pref['Hunt Code'] = [x if not pd.isnull(x) else y for x,y in zip(All_Min_Pref['Hunt Code'],All_Min_Pref['Column1'])]

del All_Min_Pref['Column1']

All_Min_Pref.to_excel('Output-Data\\Minimum-Preference-Points\\All-Min-Pref-Points.xlsx')